In [2]:
import OpenEXR
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from math import *
from tqdm.auto import tqdm, trange
from tqdm.contrib import itertools as ti
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

matplotlib.rcParams['figure.figsize'] = (20, 10)
matplotlib.rcParams['xtick.top'] = False
matplotlib.rcParams['ytick.right'] = False

def n_traces(traces):
  fig = make_subplots(1, len(traces))
  for i, trace in enumerate(traces):
    fig.add_trace(trace, 1, 1+i)
  fig.update_layout(margin=dict(t=0,b=0,l=0,r=0))
  fig.update_yaxes(autorange='reversed')
  return fig

In [3]:
def get_channel(exr, channel):
  raw_bytes = exr.channel(channel)
  v = np.frombuffer(raw_bytes, dtype=np.float32)
  window = exr.header()['displayWindow']
  height = window.max.y - window.min.y + 1
  width  = window.max.x - window.min.x + 1
  v = np.reshape(v, (height, width))
  #v = v[1100:1400,900:1100]
  return v

def get_normals(exr):
  x, y, z = (get_channel(exr, f'Ns.{A}') for A in 'XYZ')
  return np.array([x, y, z]).transpose((1, 2, 0))

def show(img):
  fig, ax = plt.subplots()
  img = ax.imshow(img)
  ax.set_aspect('auto')
  plt.colorbar(img)
  plt.show()

def showN(imgs, vmin=0, vmax=1):
  fig, axes = plt.subplots(1, len(imgs))
  if len(imgs) == 1:
    axes = [axes]
  for ax, img in zip(axes, imgs):
    pos = ax.imshow(img, vmin=vmin, vmax=vmax)
    ax.set_aspect('auto')
    fig.colorbar(pos, ax=ax)
  plt.show()

gbuf_exr = OpenEXR.InputFile('../exr/bistro_cropped.exr')
normals = get_normals(gbuf_exr)

color = get_channel(gbuf_exr, 'R')
albedo = get_channel(gbuf_exr, 'Albedo.R')
z = color / np.clip(albedo, a_min=0.01, a_max=inf)

#n_traces([go.Heatmap(z=color), go.Heatmap(z=albedo), go.Heatmap(z=z)]).show()
n_traces([go.Image(z=(normals * 0.5 + 0.5) * 255)])

In [ ]:
plt.imshow(normals * 0.5 + 0.5)

In [ ]:
RADIUS = 3
SIDE = 1 + 2 * RADIUS

filtered = np.zeros_like(color)
breaks = np.zeros_like(color, dtype=np.int8)

def rotate_ij(direction, i, j):
  match direction:
    case 0: return (i, j)
    case 1: return (j, -i)
    case 2: return (-i, -j)
    case 3: return (-j, i)

DN_THRESHOLD = 1.01

def run_in_direction(x,y, direction):
  value = 0.0
  weight = 0.0

  norigin = normals[x,y]
  nprev = norigin
  ndotprev = -inf

  for i in range(1, RADIUS+1):
    for j in range(-i, +i):
      dx, dy = rotate_ij(direction, i, j)
      xx, yy = x+dx, y+dy
      add = 1.0
      nhere = normals[xx,yy]

      dist2 = i*i + j*j
      add *= np.exp(dist2 * (-1.0 / (2 * RADIUS + 1))) # space

      ndot = np.dot(nprev, nhere)
      if ndot < 0.6 or (i > 1 and
                        (ndot > ndotprev * DN_THRESHOLD or
                         ndotprev > ndot * DN_THRESHOLD)):
        breaks[x,y] += 1
        return value, weight

      value += z[xx,yy] * add
      weight += add

      if j == 0:
        nprev = nhere
        ndotprev = ndot

  return value, weight

zw = 0
for y in trange(RADIUS, color.shape[1]-RADIUS):
  for x in range(RADIUS, color.shape[0]-RADIUS):
    # filter each pixel
    value = z[x,y]
    weight = 1.0

    # go in 4 directions
    for direction in range(4):
      vv, ww = run_in_direction(x,y, direction)
      value += vv
      weight += ww

    if weight == 1.0:
      zw += 1
    filtered[x,y] = value / weight

filtered *= albedo
print(f'zeros: {zw}/{z.shape[0] * z.shape[1]}')

In [ ]:
n_traces([go.Heatmap(z=color), go.Heatmap(z=filtered)]).show()
n_traces([go.Heatmap(z=breaks), go.Image(z=(normals * 0.5 + 0.5) * 255)]).show()

In [ ]:
RADIUS = 3
SIDE = 1 + 2 * RADIUS

filtered = np.zeros_like(color)

for y in trange(RADIUS, color.shape[1]-RADIUS):
  for x in range(RADIUS, color.shape[0]-RADIUS):
    value = 0
    ss = 0
    for dy in range(-RADIUS, RADIUS+1):
      for dx in range(-RADIUS, RADIUS+1):
        xx, yy = x + dx, y + dy
        add = 1
        add *= np.exp((dx ** 2 + dy ** 2) * (-1.0 / (2 * RADIUS + 1))) # space
        add *= np.exp(((z[xx, yy] - z[x, y])**2) * (-1.0 / 12.0)) # intensity
        add *= np.exp(((color[xx, yy] - color[x, y])**2) * (-1.0 / 1.0)) # color
        #ncross = np.cross(normals[xx,yy], normals[x,y])
        #ncross = ncross.dot(ncross)
        #add *= gauss(RADIUS * ncross, 0.1) # normal

        value += add * z[xx, yy]
        ss += add

    filtered[x,y] = value / ss

filtered *= albedo